In [2]:
!cd ..; python3 setup.py develop

/usr/local/lib/python3.11/site-packages/setuptools/_distutils/dist.py:262: UserWarning: Unknown distribution option: 'authors'
  warnings.warn(msg)
/usr/local/lib/python3.11/site-packages/setuptools/_distutils/dist.py:262: UserWarning: Unknown distribution option: 'author_emails'
  warnings.warn(msg)
/usr/local/lib/python3.11/site-packages/setuptools/dist.py:544: UserWarning: The version specified ('') is an invalid version, this may not work as expected with newer versions of setuptools, pip, and PyPI. Please see PEP 440 for more details.
  warnings.warn(
running develop
/usr/local/lib/python3.11/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
ru

In [3]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import matplotlib.pyplot as plt

from fastpathplanning.boxes import Box, BoxCollection
# from planning import build_min_distance

ModuleNotFoundError: No module named 'fastpathplanning'

# Preprocessing

In [ ]:
from time import time

d = 2
n = 5
sides = [2, .5]

boxes = [
    Box([   6.25,  4], [ 7.25,    5.75]),
    Box([ 5.5,  2.5], [ 7.5, 4.75]),
    Box([   1,    4], [   6,    5]),
    Box([   .25, 1.5], [7,    3]),
    Box([   2.25,    .75], [ 3, 2.5]),
    Box([  .5,  -.25], [ 1.5,  4.5]),
    Box([   2, 0], [   6.25,    1]),
    Box([ 4.75,  -.25], [   6, 3.75]),
    Box([ 0,  5.2], [   7, 6]),
]
B = BoxCollection(boxes)
G = B.line_graph('optimize')

In [ ]:
plt.figure()
B.plot2d(alpha=.5)
G.plot()

# Discrete shortest path

In [ ]:
start = np.array([1, .25])
goal = np.array([.5, 5.6])

In [ ]:
# Solves all-to-one shortest path for a fixed goal location.
discrete_planner, runtime = G.shortest_path(goal)
runtime

In [ ]:
# Given start location finds box sequence (and initial trajectory).
box_seq, length, runtime = discrete_planner(start)
length, runtime

# Continuous minimum-distance problem

In [ ]:
# Builds optimization problem (can be done offline).
max_boxes = len(box_seq)
continuous_planner = build_min_distance(B, max_boxes)

In [ ]:
traj, length, solver_time, runtime = continuous_planner(box_seq, start, goal)
length, solver_time, runtime

In [ ]:
plt.figure(figsize=(5, 5))
B.plot2d(alpha=.5)
plt.plot(*traj.T, c='b')

In [ ]:
# Builds optimization problem (can be done offline).
iterative = True
max_boxes = 2 * n
continuous_planner = build_min_distance(B, max_boxes, iterative, solver='ECOS')

# Refresh discrete plan.
box_seq, runtime, length = discrete_planner(start)

In [ ]:
box_seq, traj, length, iters, solver_time, runtime = continuous_planner(box_seq, start, goal)
length, solver_time, runtime

In [ ]:
plt.figure(figsize=(5, 5))
B.plot2d(alpha=.5)
B.plot2d(subset=box_seq, fc='None', ec='r')
plt.plot(*traj.T, c='b', marker='+')

# Smooth using Bezier

In [ ]:
import cvxpy as cp
from bezier import optimize_bezier, optimize_bezier_with_retiming

L = np.array([B.boxes[i].l for i in box_seq])
U = np.array([B.boxes[i].u for i in box_seq])

alpha = {2: 1, 3: 1}
initial = {0: start}
final = {0: goal}

T = n
duations = np.linalg.norm(traj[1:] - traj[:-1], axis=1)
duations *= T / sum(duations)

In [ ]:
path, sol_stats = optimize_bezier_with_retiming(L, U, duations, alpha, initial, final, verbose=True)

In [ ]:
plt.figure()
B.plot2d(alpha=.5)
path.plot(c='b', linewidth=3)
plt.axis('off')
plt.savefig('example_small.pdf', bbox_inches='tight')